In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from astropy.modeling import fitting, models
from scipy.integrate import quadrature,trapz
import seaborn as sb
import os
from io import BytesIO as bytesio
from ipywidgets import widgets
import ipywidgets as ipw

sb.set_style('dark')
matplotlib.rcParams['font.size']=12
matplotlib.rcParams['figure.figsize']=(10,10)

# **Exposure Time Calculator**

In [2]:
def coeff_calc(x0,xn,x=None,y=None,mode = None):
    if mode == 'Gaussian':
        model = models.Gaussian1D(mean = (x0+xn)*0.5, stddev = xn-x0)
        int_y,err = quadrature(model,x0,xn)
        return int_y/(xn-x0)
    else :
        return trapz(y,x)/(xn-x0)

In [3]:
def exposure_time(det_params,M,SNR):
    wavelength     = det_params['wavelength']
    bandwidth      = det_params['bandwidth']
    effective_area = det_params['effective_area']
    M_sky          = det_params['sky_brightness']
    plate_scale    = det_params['plate_scale']
    aperture       = det_params['aperture']
    dark_current   = det_params['dark_current']
    read_noise     = det_params['read_noise']

    F_0_p   = 1.51e3*(bandwidth/wavelength)*3631*effective_area
    F_m_p   = F_0_p*pow(10,-0.4*M)
    M_sky_p = M_sky - 2.5*np.log10(plate_scale**2)
    F_sky_p = F_0_p*pow(10,-0.4*M_sky_p)

    n_pix   = np.pi*((0.5*aperture)/plate_scale)**2

    A =  (F_m_p/SNR)**2
    B = -(F_m_p + F_sky_p*n_pix + dark_current*n_pix)
    C = -n_pix*(read_noise)**2

    t1 = (-B + np.sqrt(B**2 - 4*A*C))/(2*A)
    t2 = (-B - np.sqrt(B**2 - 4*A*C))/(2*A)
    t = np.where(t1>0,t1/60,t2/60)

    return t

In [41]:
df = pd.read_csv('data/detector_parameters.csv')

In [5]:
def pixel_scale(src):
    choices['telescope'] = src
    return str(df[df['telescope']==src]['plate_scale'].values[0])

In [6]:
def filters(src):
    return df[df['telescope']==src]['filter'].values

In [7]:
def bandwidth(src):
    choices['filter'] = src
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['bandwidth'].values[0])

In [8]:
def eff_lambda(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['wavelength'].values[0])

In [9]:
def eff_area(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['effective_area'].values[0])

In [10]:
def aperture(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['aperture'].values[0])

In [11]:
def rn(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['read_noise'].values[0])

In [12]:
def dc(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['dark_current'].values[0])

In [13]:
def skt_bri(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['sky_brightness'].values[0])

In [40]:
global choices
choices = {}
choices['coeff'] = None

h0 = widgets.HTML("<h1>Instrument</h1>")
display(h0)

l2 = widgets.HTML("<h fontsize=10>Use CSV file for profiles with columns 'wav', 'response'</h>")
display(l2)
c1 = []
c1.append(widgets.HTML("<h2>Telescope</h2>"))
c1.append(widgets.Label("Select Telescope"))

c1.append(widgets.Dropdown(
    options=df['telescope'].unique(),
    value='INSIST'))

c1.append(widgets.HTML('<h>Diameter of primary (cm)</h>'))
c1.append(widgets.Text("50"))

c1.append(widgets.HTML('<h>Plate Scale (\"/pixels)</h>'))
c1.append(widgets.Text())

c1.append(widgets.HTML('<h>No of reflecting surfaces</h>'))
c1.append(widgets.Text("5"))

c1.append(widgets.HTML('<h>Coating coefficient</h>'))
c1.append(widgets.Text("0.95"))

c1.append(widgets.HTML("<h fontsize=10>Custom coating coefficient</h>"))
c1.append(widgets.FileUpload(multiple=True))

ipw.dlink((c1[2],'value'), (c1[6],'value'), transform = pixel_scale)

v1 = widgets.VBox(c1)

#----------------------------------------------------------

c2 = []

c2.append(widgets.HTML("<h2>Filter<h2>"))
c2.append(widgets.Label("Select Filter :"))

c2.append(widgets.Dropdown(
    options=df[df['telescope']==c1[2].value]['filter'].unique(),
    value='NUV'))

ipw.dlink((c1[2],'value'), (c2[2],'options'), transform = filters)

c2.append(widgets.Label(r"$\Delta\lambda$ (Å)"))
c2.append(widgets.Text())

c2.append(widgets.Label(r"$\lambda_{eff}$ (Å)"))
c2.append(widgets.Text())

c2.append(widgets.Label("Transmission Coefficient"))
c2.append(widgets.Text("0.9"))

c2.append(widgets.Label("Sky Brightness"))
c2.append(widgets.Text())

c2.append(widgets.Label(r"Effective area $(cm^2)$"))
c2.append(widgets.Text("0.9"))

c2.append(widgets.HTML("<h fontsize=10>Custom filter Response functions</h>"))
c2.append(widgets.FileUpload(multiple=True))
c2.append(widgets.Button(description="Recalculate"))

ipw.dlink((c2[2],'value'), (c2[4],'value'), transform = bandwidth)
ipw.dlink((c2[2],'value'), (c2[6],'value'), transform = eff_lambda)
ipw.dlink((c2[2],'value'), (c2[10],'value'), transform = skt_bri)
ipw.dlink((c2[2],'value'), (c2[12],'value'), transform = eff_area)

v2 = widgets.VBox(c2)

#----------------------------------------------------------

c3 = []

c3.append(widgets.HTML("<h2>Detector</h2>"))
c3.append(widgets.Label("Photometric aperture (arcsec)"))
c3.append(widgets.Text())

ipw.dlink((c2[2],'value'), (c3[2],'value'), transform = aperture)

c3.append(widgets.Label(r"Quantum efficiency"))
c3.append(widgets.Text("0.95"))

c3.append(widgets.Label("Read Noise"))
c3.append(widgets.Text())

c3.append(widgets.Label("Dark Current"))
c3.append(widgets.Text())

c3.append(widgets.HTML("<h fontsize=10>Custom Quantum Efficiency profile</h>"))
c3.append(widgets.FileUpload(multiple=True))

ipw.dlink((c2[2],'value'), (c3[6],'value'), transform = rn)
ipw.dlink((c2[2],'value'), (c3[8],'value'), transform = dc)

v3 = widgets.VBox(c3)

h = widgets.HBox([v1, v2,v3])

display(h)

h0 = widgets.HTML("<h1>Source</h1>")
display(h0)

l1 = widgets.Label("Select calculator mode")
display(l1)

mode = widgets.Dropdown(
    options=['Time','SNR'],
    value='Time'
)
display(mode)

l2 = widgets.Label("Enter AB magnitude of the source")
display(l2)
Mag = widgets.Text("10")
display(Mag)

btn = widgets.Button(description="Submit")

output = widgets.Output()

@output.capture(clear_output=True,wait=True)

def recalc(b):
    # Telescope coefficients
    if len(c1[-1].value)> 0 :
        coeff_t = []
        for file in c1[-1].value:
            df_coff = pd.read_csv(bytesio(c1[-1].value[file]['content']))
            coeff_t.append(coeff_calc(df_coeff['wav'].min(),df_coeff['wav'].max(),x=df_coeff['wav'].values,y=df_coeff['response'].values))
        coeff_t = np.array(coeff_t).prod()
        no_refl_surf = float(c[8].value)

        coeff_t = pow(coeff_t,no_refl_surf)

    elif c1[10].value.replace('.','',1).isnumeric()==True and float(c1[10].value)>0 and c1[8].value.replace('.','',1).isnumeric()==True:
        coeff_t = float(c1[10].value)
        no_refl_surf = float(c1[8].value)
        coeff_t = pow(coeff_t,no_refl_surf)
        
    # Filter Coefficients

    if len(c2[-2].value)> 0 :
        coeff_t = []
        for file in c1[-1].value:
            df_coff = pd.read_csv(bytesio(c1[-1].value[file]['content']))
            coeff_f.append(coeff_calc(df_coeff['wav'].min(),df_coeff['wav'].max(),x=df_coeff['wav'].values,y=df_coeff['response'].values))

        coeff_f = np.array(coeff_f).prod()

    elif c2[8].value.replace('.','',1).isnumeric()==True :
        if float(c2[8].value)>0:
            coeff_f = float(c2[8].value)

    # Detecter Coefficients
    if len(c3[-1].value)> 0 :
        coeff_d = []
        for file in c1[-1].value:
            df_coff = pd.read_csv(bytesio(c1[-1].value[file]['content']))
            coeff_d.append(coeff_calc(df_coeff['wav'].min(),df_coeff['wav'].max(),x=df_coeff['wav'].values,y=df_coeff['response'].values))  
        coeff_d = np.array(coeff_d).prod()
    elif c3[4].value.replace('.','',1).isnumeric()==True :
        if float(c3[4].value)>0:
            coeff_d = float(c3[4].value)
    
    l = widgets.Label(f"Calculated Telescope, filter and detector coefficients from given input file\n are {np.round(coeff_t,3)}, {np.round(coeff_f,3)} and {np.round(coeff_d,3)} respectively")
    display(l)
    
    choices['coeff'] = coeff_t*coeff_f*coeff_d 
    
@output.capture(clear_output=True,wait=True)
        
def submit(b):  
    if mode.value=='Time':
        l2 = widgets.Label("Enter required SNR")
        display(l2)
        SNR = widgets.Text("3")
        display(SNR)
        if len(Mag.value)<1:
            Mag.value = '22'
            out = widgets.Label(f"Default AB magnitude set : 22")
            display(out)
        if len(SNR.value)<1:
            SNR.value = '5'
            out = widgets.Label(f"Default SNR set : 5")
            display(out)
        
        if float(Mag.value)>0 and float(SNR.value)>0 :
            temp = df[ (df['telescope']==choices['telescope']) & (df['filter']==choices['filter'])]
            
            global det_params

            det_params = choices
            det_params['wavelength']     = float(c2[6].value) if c2[6].value.replace('.','',1).isnumeric()==True else temp['wavelength'].values[0]
            det_params['bandwidth']      = float(c2[4].value) if c2[4].value.replace('.','',1).isnumeric()==True else temp['bandwidth'].values[0]
            
            if c1[4].value.replace('.','',1).isnumeric()==True and choices['coeff'] is not None:
                det_params['effective_area'] = choices['coeff']*0.25*np.pi*float(c1[4].value)**2  
            else :
                det_params['effective_area'] = temp['effective_area'].values[0]
                
            det_params['sky_brightness'] = float(c2[10].value) if c2[10].value.replace('.','',1).isnumeric()==True else temp['sky_brightness'].values[0]
            det_params['plate_scale']    = float(c1[6].value) if c1[6].value.replace('.','',1).isnumeric()==True else temp['pixel_scale'].values[0]
            det_params['aperture']       = float(c3[2].value) if c3[2].value.replace('.','',1).isnumeric()==True else temp['aperture'].values[0]
            det_params['read_noise']     = float(c3[6].value) if c3[6].value.replace('.','',1).isnumeric()==True else temp['read_noise'].values[0]
            det_params['dark_current']   = float(c3[8].value) if c3[8].value.replace('.','',1).isnumeric()==True else temp['dark_current'].values[0]
            
            t = exposure_time(det_params,float(Mag.value),float(SNR.value))
            out = widgets.Label(f"Required exposure time for observing source with magnitude {Mag.value} and SNR {SNR.value} = {np.round(t,5)} minutes")
            display(out)
            l1 = widgets.Label(f"Effective Area used in calclation : {np.round(det_params['effective_area'],3)} cm2")
            display(l1)
        else:
            out = widgets.Label("AB magnitude and SNR should be greater than 0")
            display(out)
display(btn)
    
btn.on_click(submit) 

c2[-1].on_click(recalc)

output

HTML(value='<h1>Instrument</h1>')

HTML(value="<h fontsize=10>Use CSV file for profiles with columns 'wav', 'response'</h>")

HTML(value='<h1>Source</h1>')

Label(value='Select calculator mode')

Dropdown(options=('Time', 'SNR'), value='Time')

Label(value='Enter AB magnitude of the source')

Text(value='10')

Button(description='Submit', style=ButtonStyle())

Output()